In [37]:
#Criando músicas do Charlie Brown Jr. Com Machine Learning
import glob
import nltk
import string
import tensorflow as tf
from tensorflow import keras
from nltk import word_tokenize
from nltk.collocations import *
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from tensorflow.keras.preprocessing.text import Tokenizer

In [30]:
files = glob.glob("musicas/charliebrownjr/*")
print('Músicas:')
print('------------------------------------------------------------------------------------------------------------------------------')
print(files)
print('------------------------------------------------------------------------------------------------------------------------------')

docs = ''
for fname in files:
    with open(fname,'r') as f:
        docs = docs +' '+ f.read()

Músicas:
------------------------------------------------------------------------------------------------------------------------------
['musicas/charliebrownjr\\cbjr-ceu-azul.txt', 'musicas/charliebrownjr\\cbjr-como-tudo-deve-ser.txt', 'musicas/charliebrownjr\\cbjr-dias-luta-dias-gloria.txt', 'musicas/charliebrownjr\\cbjr-lugar-ao-sol.txt', 'musicas/charliebrownjr\\cbjr-meu-novo-mundo.txt', 'musicas/charliebrownjr\\cbjr-pontes-indestrutiveis.txt', 'musicas/charliebrownjr\\cbjr-proibida-pra-mim.txt', 'musicas/charliebrownjr\\cbjr-senhor-do-tempo.txt', 'musicas/charliebrownjr\\cbjr-so-os-loucos-sabem.txt', 'musicas/charliebrownjr\\cbjr-zoio-de-lula.txt']
------------------------------------------------------------------------------------------------------------------------------


In [41]:
#divindo as canções em versos
versos = docs.lower().split('\n')
print(versos)

[' ouvir ceu azul', 'nao encontramos nada.', 'tao natural quanto a luz do dia', 'mas que preguica boa', 'me deixa aqui a toa', 'hoje ninguem vai estragar meu dia', 'so vou gastar energia pra beijar sua boca', '', 'fica comigo entao', 'nao me abandona, nao', 'alguem te perguntou como e que foi seu dia?', 'uma palavra amiga, uma noticia boa', 'isso faz falta no dia a dia', 'a gente nunca sabe quem sao essas pessoas', '', 'eu so queria te lembrar', 'que aquele tempo eu nao podia fazer mais por nos', 'eu estava errado e voce nao tem que me perdoar', '', 'mas tambem quero te mostrar', 'que existe um lado bom nessa historia', 'tudo que ainda temos a compartilhar', '', 'e viver', 'e cantar', 'nao importa qual seja o dia', 'vamos viver', 'vadiar', 'o que importa e nossa alegria', '', 'vamos viver', 'e cantar', 'nao importa qual seja o dia', 'vamos viver', 'vadiar', 'o que importa e nossa alegria', '', 'tao natural quanto a luz do dia', 'mas que preguica boa', 'me deixa aqui a toa', 'hoje ningu

In [31]:
#Pré-Processamento

words = nltk.word_tokenize(docs) #dividindo todas as letras de músicas e separando em palavras
words = [w.lower() for w in words if w.isalpha() and w not in string.ascii_letters] #removendo símbolos e caracteres

#removendo stop_words - palavras que não contribuem para a semântica
stop_words = stopwords.words('portuguese')
words = [w for w in words if w not in stop_words] 

#identificando n-gramas
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_words(words)
for i in finder.nbest(bigram_measures.raw_freq,20):
    words.append(i[0].lower() + ' ' + i[1].lower())

words = [WordNetLemmatizer().lemmatize(w,'v') for w in words] #normalização léxica com lematização - converter sua palavra para sua forma base - andando, andares -> andar
    
print(words)

['ouvir', 'ceu', 'azul', 'nao', 'encontramos', 'nada', 'tao', 'natural', 'quanto', 'luz', 'dia', 'preguica', 'boa', 'deixa', 'aqui', 'toa', 'hoje', 'ninguem', 'vai', 'estragar', 'dia', 'so', 'vou', 'gastar', 'energia', 'pra', 'beijar', 'boca', 'fica', 'comigo', 'entao', 'nao', 'abandona', 'nao', 'alguem', 'perguntou', 'dia', 'palavra', 'amiga', 'noticia', 'boa', 'faz', 'falta', 'dia', 'dia', 'gente', 'nunca', 'sabe', 'sao', 'pessoas', 'so', 'queria', 'lembrar', 'tempo', 'nao', 'podia', 'fazer', 'errado', 'voce', 'nao', 'perdoar', 'tambem', 'quero', 'mostrar', 'existe', 'lado', 'bom', 'nessa', 'historia', 'tudo', 'ainda', 'compartilhar', 'viver', 'cantar', 'nao', 'importa', 'dia', 'vamos', 'viver', 'vadiar', 'importa', 'alegria', 'vamos', 'viver', 'cantar', 'nao', 'importa', 'dia', 'vamos', 'viver', 'vadiar', 'importa', 'alegria', 'tao', 'natural', 'quanto', 'luz', 'dia', 'preguica', 'boa', 'deixa', 'aqui', 'toa', 'hoje', 'ninguem', 'vai', 'estragar', 'dia', 'so', 'vou', 'gastar', 'ener

In [36]:
#modelo básico 1 - LSTM - Long Short-Term Memory layer - Hochreiter 1997.
LSTM = keras.models.Sequential()
LSTM.add(keras.layers.Embedding(words,100,input_length=))
LSTM.add(keras.layers.LSTM(150,return_sequences=True,activation='relu',input_shape=(1,1)))
LSTM.add(keras.layers.LSTM(100,return_sequences=True,activation='relu'))
LSTM.add(keras.layers.Dropout(0.2))
LSTM.add(keras.layers.Dense(1, activation='softmax'))
LSTM.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 1, 150)            91200     
_________________________________________________________________
lstm_5 (LSTM)                (None, 1, 100)            100400    
_________________________________________________________________
dropout_2 (Dropout)          (None, 1, 100)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 1, 1)              101       
Total params: 191,701
Trainable params: 191,701
Non-trainable params: 0
_________________________________________________________________
